# 환경설정

In [2]:
import os
from modules import logging
from dotenv import load_dotenv
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [3]:
# langsmith logging
logging.langsmith("Model_RAG")

# API KEY 정보로드
load_dotenv()

LangSmith 추적을 시작합니다.
[프로젝트명]
Model_RAG


True

In [4]:
# check metadata
def show_metadata(docs):
    if docs:
        print("[metadata]")
        print(list(docs[0].metadata.keys()))
        print("\n[examples]")
        max_key_length = max(len(k) for k in docs[0].metadata.keys())
        for k, v in docs[0].metadata.items():
            print(f"{k:<{max_key_length}} : {v}")

# 파일로드

In [5]:
FILE_PATH = "data\무배당 메리츠 다이렉트 운전자보험2409.pdf"
raw_documents = PyMuPDFLoader(FILE_PATH).load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=125)
documents = text_splitter.split_documents(raw_documents)

In [6]:
show_metadata(documents)

[metadata]
['source', 'file_path', 'page', 'total_pages', 'format', 'title', 'author', 'subject', 'keywords', 'creator', 'producer', 'creationDate', 'modDate', 'trapped']

[examples]
source       : data\무배당 메리츠 다이렉트 운전자보험2409.pdf
file_path    : data\무배당 메리츠 다이렉트 운전자보험2409.pdf
page         : 0
total_pages  : 13
format       : PDF 1.4
title        : 
author       : 
subject      : 
keywords     : 
creator      : Hancom PDF 1.3.0.430
producer     : Hancom PDF 1.3.0.430
creationDate : D:20241111165015+09'00'
modDate      : D:20241111165015+09'00'
trapped      : 


In [32]:
print(f"분할된 청크의수: {len(documents)}")

분할된 청크의수: 51


# 임베딩

In [7]:
ollama_embeddings = OllamaEmbeddings(
    model="nomic-embed-text"
)

C:\Users\vkxql\AppData\Local\Temp\ipykernel_8952\1667557180.py:1: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(


In [34]:
vectorstore = FAISS.from_documents(documents=documents, embedding=ollama_embeddings)

In [35]:
# 로컬 Disk 에 저장
vectorstore.save_local(folder_path="faiss_db", index_name="meritz_index")

# 파일 불러오기

In [8]:
loaded_db = FAISS.load_local(
    folder_path="faiss_db",
    index_name="meritz_index",
    embeddings=ollama_embeddings,
    allow_dangerous_deserialization=True,
)

In [10]:
print(loaded_db.index_to_docstore_id[3])

06c146f3-15d7-40e3-8e23-e69f81ac3507


In [11]:
retriever = loaded_db.as_retriever()

In [12]:
retriever.invoke("보험")

[Document(metadata={'source': 'data\\무배당 메리츠 다이렉트 운전자보험2409.pdf', 'file_path': 'data\\무배당 메리츠 다이렉트 운전자보험2409.pdf', 'page': 3, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Hancom PDF 1.3.0.430', 'producer': 'Hancom PDF 1.3.0.430', 'creationDate': "D:20241111165015+09'00'", 'modDate': "D:20241111165015+09'00'", 'trapped': ''}, page_content='(1) 보험금지급사유 및 지급금액\n○ 기본계약\n보장명\n지급사유\n지급금액\n운전자용 교통상해사망/\n비운전자용 교통상해사망\n보험기간 중 교통상해(비운전자용은 비운전중 교통상해로 한\n함)의 직접결과로써 사망한 경우(질병으로 인한 사망은 제외\n합니다)\n가입금액\n○ 선택계약\n[상해]'),
 Document(metadata={'source': 'data\\무배당 메리츠 다이렉트 운전자보험2409.pdf', 'file_path': 'data\\무배당 메리츠 다이렉트 운전자보험2409.pdf', 'page': 7, 'total_pages': 13, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Hancom PDF 1.3.0.430', 'producer': 'Hancom PDF 1.3.0.430', 'creationDate': "D:20241111165015+09'00'", 'modDate': "D:20241111165015+09'00'", 'trapped': ''}, page_content='배우자, 자녀 제외)에게 아래의 상해